In [22]:
pip install --upgrade gcsfs fsspec

  Using cached fsspec-2024.12.0-py3-none-any.whl.metadata (11 kB)
Using cached fsspec-2024.12.0-py3-none-any.whl (183 kB)
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.9.0
    Uninstalling fsspec-2024.9.0:
      Successfully uninstalled fsspec-2024.9.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.10.1 requires pyarrow<18.0.0a0,>=14.0.0, but you have pyarrow 11.0.0 which is incompatible.
dask 2024.10.0 requires cloudpickle>=3.0.0, but you have cloudpickle 2.2.1 which is incompatible.
datasets 3.2.0 requires fsspec[http]<=2024.9.0,>=2023.1.0, but you have fsspec 2024.12.0 which is incompatible.
datasets 3.2.0 requires pyarrow>=15.0.0, but you have pyarrow 11.0.0 which is incompatible.


In [1]:
!pip3 install -qU \
langchain \
tiktoken \
datasets \
pinecone-client
!pip3 install protobuf==3.20.3
!pip3 install apache-beam==2.50.0
!pip install fastapi uvicorn nest_asyncio pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 MB 10.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-beam 2.50.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.50.0 requires pyarrow<12.0.0,>=3.0.0, but you have pyarrow 18.1.0 which is incompatible.
cudf-cu12 24.10.1 requires pyarrow<18.0.0a0,>=14.0.0, but you have pyarrow 18.1.0 which is incompatible.
pylibcudf-cu12 24.10.1 requires pyarrow<18.0.0a0,>=14.0.0, but you have pyarrow 18.1.0 which is incompatible.
  Using cached dill-0.3.1.1-py3-none-any.whl
  Using cached pyarrow-11.0.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.0 kB)
Using cached pyarrow-11.0.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (34.9 MB)
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
  

In [2]:
import os
import pinecone
from pinecone import Pinecone
from pinecone import ServerlessSpec
import time
from datasets import load_dataset
import tiktoken
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema.document import Document
import pandas as pd
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM
import torch
from tqdm.auto import tqdm
from uuid import uuid4

In [3]:
api_ke = os.environ.get("PINECONE_API_KEY")
pc = Pinecone(api_key='PINECONE_API_KEY')

In [4]:
cloud = os.environ.get('PINECONE_CLOUD') or 'aws'
region = os.environ.get('PINECONE_REGION') or 'us-east-1'

spec = ServerlessSpec(cloud=cloud, region=region)
index_name ='rag-index'
if index_name in pc.list_indexes().names():
  pc.delete_index(index_name)
# does not exist, create index
pc.create_index(
    index_name,
    dimension=384,  # dimensionality of sentence transformer
    metric='cosine',
    spec=spec
)
# connect to index
index = pc.Index(index_name)

In [5]:
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [6]:
wiki_en = load_dataset("wikipedia", "20220301.en", split="train[:10]")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/16.0k [00:00<?, ?B/s]

wikipedia.py:   0%|          | 0.00/36.7k [00:00<?, ?B/s]

The repository for wikipedia contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/wikipedia.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


train-00000-of-00041.parquet:   0%|          | 0.00/1.04G [00:00<?, ?B/s]

...

train-00037-of-00041.parquet:   0%|          | 0.00/195M [00:00<?, ?B/s]

train-00038-of-00041.parquet:   0%|          | 0.00/203M [00:00<?, ?B/s]

train-00039-of-00041.parquet:   0%|          | 0.00/192M [00:00<?, ?B/s]

train-00040-of-00041.parquet:   0%|          | 0.00/185M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/6458670 [00:00<?, ? examples/s]

In [7]:
wiki_en.to_pandas().head()

,id,url,title,text
0,12,https://en.wikipedia.org/wiki/Anarchism,Anarchism,Anarchism is a political philosophy and moveme...
1,25,https://en.wikipedia.org/wiki/Autism,Autism,Autism is a neurodevelopmental disorder charac...
2,39,https://en.wikipedia.org/wiki/Albedo,Albedo,Albedo (; ) is the measure of the diffuse refl...
3,290,https://en.wikipedia.org/wiki/A,A,"A, or a, is the first letter and the first vow..."
4,303,https://en.wikipedia.org/wiki/Alabama,Alabama,Alabama () is a state in the Southeastern regi...


In [8]:
model_name = 'sentence-transformers/all-MiniLM-L6-v2' #Hugging Face Model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)
def get_embedding(text):
  encoded_input = tokenizer(text, padding=True, truncation=True, return_tensors='pt')

  with torch.no_grad():
      model_output = model(**encoded_input)

  embedding = model_output.last_hidden_state[0].mean(dim=0).squeeze().tolist()
  # if len(embedding) > 1536:
  #       embedding = embedding[:1536]
  # elif len(embedding) < 1536:
  #     embedding = embedding + [0.0] * (1536 - len(embedding))

  return embedding

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

In [9]:
tiktoken.encoding_for_model('text-embedding-ada-002')
enc_tokenizer = tiktoken.get_encoding('cl100k_base')

# Create our custom tiktoken function
def tiktoken_len(text: str) -> int:


    tokens = enc_tokenizer.encode(
        text,
    )
    return len(tokens)

In [10]:
def chunk_by_size(text: str, size: int = 50) -> list[Document]:

    text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = size,
    chunk_overlap = 20,
    length_function = tiktoken_len,
    add_start_index = True,
)
    return text_splitter.create_documents([text])

In [11]:
def create_chunks_metadata_embeddings(dataset: pd.DataFrame) -> list[dict]:
    """
    Given a dataset, split text data into chunks, extract metadata, create embeddings for each chunk.

    :param dataset: Data we want to process.
    :return: List of data objects to upsert into our Pinecone index.
    """
    data_objs = []

    # For each row in our dataset:
    for index, row in tqdm(dataset.iterrows()):  # (tqdm library prints status of for-loop to console)
        # Create chunks
        chunked_text = chunk_by_size(row["text"])

        # Extract just the string content from the chunk
        chunked_text = [c.page_content for c in chunked_text]

        # Extract some metadata, create an ID, and generate an embedding for the chunk.
        # Wrap that all in a dictionary, and append that dictionary to a list (`data_objs`).
        for idx, text in enumerate(chunked_text):
            payload = {
                "metadata": {
                    "url": row["url"],
                    "title": row["title"],
                    "chunk_num": idx,
                    "text_content": text
                },
             "id": str(uuid4()),
            "values":get_embedding(text)
            }
            data_objs.append(payload)

    # Return list of dictionaries, each containing our metadata, ID, and embedding, per chunk.
    return data_objs

In [12]:
data_objs_en = create_chunks_metadata_embeddings(wiki_en.to_pandas().head(3))

0it [00:00, ?it/s]

In [13]:
len(data_objs_en)

737

In [14]:
data_objs_en[0]


{'metadata': {'url': 'https://en.wikipedia.org/wiki/Anarchism',
  'title': 'Anarchism',
  'chunk_num': 0,
  'text_content': 'Anarchism is a political philosophy and movement that is sceptical of authority and rejects all involuntary, coercive forms of hierarchy. Anarchism calls for the abolition of the state, which it holds to be unnecessary, undesirable, and harmful. As'},
 'id': '079f985f-cb9e-4c98-b50a-4435f303ee11',
 'values': [0.00446190545335412,
  -0.011423846706748009,
  -0.025705955922603607,
  -0.026579948142170906,
  0.04562781751155853,
  -0.048472922295331955,
  0.015663200989365578,
  -0.03178687393665314,
  -0.0750642642378807,
  0.1933964043855667,
  -0.013172253966331482,
  0.023438815027475357,
  0.07216513156890869,
  -0.02339436113834381,
  -0.20954342186450958,
  0.19680815935134888,
  -0.11812631040811539,
  -0.15838825702667236,
  -0.002150018000975251,
  0.19155985116958618,
  -0.028574468567967415,
  -0.11914882808923721,
  -0.131313294172287,
  0.2969968914985

In [15]:
BATCH_SIZE = 100

def batch_upsert(data: list[dict], index: pinecone.Index):
    """
    Upsert data objects to a Pinecone index in batches.

    :param data: Data objects we want to upsert.
    :param index: Index into which we want to upsert our data objects.
    :namespace: Namespace within our index into which we want to upsert our data objects.
    """
    for i in range(0, len(data), BATCH_SIZE):
        batch = data[i:i+BATCH_SIZE]
        # print(batch)
        index.upsert(vectors=batch)

In [16]:
batch_upsert(data_objs_en, index)

In [17]:
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

!pip install fastapi uvicorn  

In [43]:
from fastapi import FastAPI

app = FastAPI()

In [48]:
import uvicorn
from pyngrok import ngrok

In [56]:
!ngrok config add-authtoken $NGROK_AUTH

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [19]:
def query_pinecone(query, top_k=3, namespace="default"):
    query_embedding = get_embedding(query)
    results = index.query(
        vector=query_embedding,
        top_k=top_k,
        namespace=namespace,
        include_metadata=True
    )
    return [index[int(match["id"])] for match in results["matches"]]

In [58]:
res_model_name = "gpt2"
res_tokenizer = AutoTokenizer.from_pretrained(res_model_name)
res_model = AutoModelForCausalLM.from_pretrained(res_model_name)
if res_tokenizer.pad_token_id is None:
    res_tokenizer.pad_token_id = res_tokenizer.eos_token_id

@app.get("/items/{query}")
def generate_response(query):
    context = " ".join(query_pinecone(query))
    prompt = f"""
    You are a knowledgeable assistant. Use the following context to answer the question:
    Context: {context}
    Question: {query}
    Answer:
    """
    # Tokenize the input prompt
    inputs = res_tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)

    # Generate a response
    outputs = res_model.generate(
        inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_length=400,  # Adjust max length for output
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        temperature=0.9,
        top_p=0.9,
        top_k=20,
        do_sample=True
    )

    # Decode the output
    response = res_tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Extract the answer part of the response
    answer_start = response.find("Answer:")
    if answer_start != -1:
        return response[answer_start + len("Answer:"):].strip()
    return response.strip()
if __name__ == "__main__":

    public_url = ngrok.connect(8000)
    print(public_url)
    uvicorn.run(app, host="127.0.0.1", port=8000)




ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-52' coro=<Server.serve() done, defined at /usr/local/lib/python3.10/dist-packages/uvicorn/server.py:68> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/main.py", line 579, in run
    server.run()
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/server.py", line 66, in run
    return asyncio.run(self.serve(sockets=sockets))
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 133, in _run_once
    handle._run()
  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "/usr/lib/python3.10/asyncio/tasks.py", 

NgrokTunnel: "https://fcac-34-168-56-79.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [18780]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     2406:b400:72:e0aa:452c:4eb2:99bd:d6d1:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     2406:b400:72:e0aa:452c:4eb2:99bd:d6d1:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     2406:b400:72:e0aa:452c:4eb2:99bd:d6d1:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     2406:b400:72:e0aa:452c:4eb2:99bd:d6d1:0 - "GET /openapi.json HTTP/1.1" 200 OK


/usr/local/lib/python3.10/dist-packages/fastapi/openapi/utils.py:225: UserWarning: Duplicate Operation ID generate_response_items__query__get for function generate_response
  warnings.warn(message, stacklevel=1)
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


INFO:     2406:b400:72:e0aa:452c:4eb2:99bd:d6d1:0 - "GET /items/what%20anarchy HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [18780]


In [59]:
query = input("Enter your query: ")
print(generate_response(query))

Enter your query: what is anarchy?


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


The word anarchy is used in relation to political and social issues.
A.  In the anarchist movement, anarchy means "the right to take control of one's own affairs by the people."
B.   Anarchy means freedom to make your own choices about how and what to do with the resources of your family. You should consider this option as part of the core of any anarchist society. When you make decisions you want to follow the will of others, and so anarchists should be very careful about not letting them take your personal decision. This means that you should never allow any of this to become part or parcel of yours. Anarchism means to work with others to bring about a common goal. In this sense, anarchism is the way of life. It means living without control or coercion and without fear of retribution. The only way you can have a life of anarchy without any restrictions is if you live in an anarchist utopia. Anarchy means the right of everyone to govern their own lives and to control their destinies. 